# WEEK 4 - FINAL PROJECT

### 15-Puzzle with Reinforcement Learning

In [29]:
import numpy as np
from collections import defaultdict, deque


In [30]:
BOARD_SIZE = 4

# Reduced goal state (only first row matters)
GOAL_STATE = (
    1, 2, 3, 4,
    -1, -1, -1, -1,
    -1, -1, -1, -1,
    -1, -1, -1, 0
)


In [31]:
ACTIONS = {
    "UP":    (-1,  0),
    "DOWN":  ( 1,  0),
    "LEFT":  ( 0, -1),
    "RIGHT": ( 0,  1)
}


In [32]:
def move(state, action):
    """
    Applies an action to a reduced puzzle state.
    Returns next state or None if action is invalid.
    """
    s = list(state)
    zero_index = s.index(0)
    r, c = divmod(zero_index, BOARD_SIZE)
    dr, dc = ACTIONS[action]

    nr, nc = r + dr, c + dc
    if nr < 0 or nr >= BOARD_SIZE or nc < 0 or nc >= BOARD_SIZE:
        return None

    new_zero = nr * BOARD_SIZE + nc
    s[zero_index], s[new_zero] = s[new_zero], s[zero_index]
    return tuple(s)


In [33]:
def generate_states(limit=60000):
    """
    Generates reduced state space using BFS from the goal.
    """
    visited = set([GOAL_STATE])
    queue = deque([GOAL_STATE])

    while queue and len(visited) < limit:
        s = queue.popleft()
        for a in ACTIONS:
            ns = move(s, a)
            if ns and ns not in visited:
                visited.add(ns)
                queue.append(ns)

    return list(visited)


In [34]:
states = generate_states()
print("Number of reduced states:", len(states))


Number of reduced states: 60000


In [35]:
TRANSITIONS = {}

for s in states:
    TRANSITIONS[s] = {}
    for a in ACTIONS:
        ns = move(s, a)
        if ns:
            TRANSITIONS[s][a] = ns


In [36]:
def reward(state):
    """
    Reward is 0 at goal, -1 otherwise.
    Encourages shortest solution.
    """
    return 0 if state == GOAL_STATE else -1


In [37]:
V = defaultdict(float)

GAMMA = 0.95      # discount factor
THETA = 1e-3     # convergence threshold
MAX_ITERS = 200  # safety cap


In [38]:
for iteration in range(MAX_ITERS):
    delta = 0
    for s in states:
        if s == GOAL_STATE:
            continue

        v_old = V[s]

        V[s] = max(
            reward(s) + GAMMA * V[ns]
            for ns in TRANSITIONS[s].values()
        )

        delta = max(delta, abs(v_old - V[s]))

    if delta < THETA:
        print(f"Value Iteration converged at iteration {iteration}")
        break


Value Iteration converged at iteration 13


In [39]:
policy = {}

for s in states:
    if s == GOAL_STATE:
        continue

    policy[s] = max(
        TRANSITIONS[s],
        key=lambda a: reward(s) + GAMMA * V[TRANSITIONS[s][a]]
    )


In [40]:
def solve(start, max_steps=100):
    """
    Follows the learned policy safely.
    Stops if policy is undefined.
    """
    s = start
    path = [s]

    for _ in range(max_steps):
        if s == GOAL_STATE:
            break
        if s not in policy:
            print("Policy missing for state, stopping early.")
            break
        s = TRANSITIONS[s][policy[s]]
        path.append(s)

    return path


In [41]:
start = GOAL_STATE

for _ in range(20):
    action = np.random.choice(list(ACTIONS))
    next_state = move(start, action)
    
    if next_state is not None:
        start = next_state


In [42]:
solution = solve(start)

print("Number of steps:", len(solution))


Number of steps: 3
